---

title: Linux UART 的使用

keywords: maixpy3, UART, Linux

desc: maixpy3 doc: UART 的使用

---


## UART 使用教程

在 Linux 系统中，串口是以设备的形式存在的，所使用的方式和原来的单片机方式有所不同。这是系统标准的 UART 通讯，和 Linux 系统中的串口操作相似。下面以 MaixII-Dock 为例子，来简单的简述一下如何使用 UART。

### 准备
通过查看开发板的管脚定义图，确定需要使用的 UART 通道。下面的代码是使用 MaixII-Dock 的 UART-1 通道

![MaixII-Dock 管脚图](./../asserts/M2Dock_pin.jpg)

### 开始

In [ ]:
import serial
ser = serial.Serial("/dev/ttyS1",115200)    # 连接串口
print('serial test start ...')              
ser.write(b"Hello Wrold !!!\n")             # 输入需要通讯的内容
while True:
    ser.setDTR(True)                        
    ser.setRTS(True)
    tmp = ser.readline()
    print(tmp)
    ser.write(tmp)
    ser.setDTR(False)
    ser.setRTS(False)

代码中的 `/dev/ttyS1` 是指串口通道1，不同的开发板，串口的表示方式不一样，请自行查看对应开发板的串口表达方式

这是标准 Python 的串口库，更多的使用查看 [Python serial](https://pypi.org/project/pyserial/)

## UART 用途

这是操作系统的标准 URAT，可以和单片机进行串口通讯，也可以对带有串口协议的设备、外设通讯。